<a href="https://colab.research.google.com/github/JimKing100/Jestimate/blob/master/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [125]:
!pip install category_encoders==2.0.0

In [0]:
# Import libraries
import pandas as pd
import numpy as np
import math

from datetime import datetime

import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression, SelectKBest

import statsmodels.api as sm

# import category_encoders as ce
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LogisticRegressionCV
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.pipeline import make_pipeline
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_absolute_error

In [127]:
# Load SF real estate data - 10 years (2009-2018) of single family home sales in San Francisco downloaded from the SF MLS
# Longitude and latitude were added to the csv file prior to loading using geocoding.geo.census.gov 
df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/Jestimate/master/data/SF-SFR-Sales-Final2d.csv')

# Rename subdistr_desc to neighborhood
df = df.rename(columns={'subdist_no': 'nid', 'subdist_desc': 'neighborhood'})

# Check the data
print(df.shape)
df.head(5)

(23711, 39)


,longitude,latitude,elevation,full_address,city,state,street_no,street_name,street_suffix,zip,area,district_no,district_desc,nid,neighborhood,on_market_date,cdom,orig_list_price,sale_date,sale_price,rooms,baths,beds,sf_source,sf_source_decs,sf,lot_acres,lot_sf,year_built,zoning,lot_desc,drive_side,parking,park_leased,num_parking,shopping,transportation,type,views
0,-122.50965,37.780280,200.83,"2645 El Camino Del Mar, San Francisco, CA 94121",San Francisco,CA,2645,El Camino Del Mar,NaN,94121,1050,1,SF District 1,1050,1 - Outer Richmond,3/14/13,4,1095000,3/22/13,1260000,8,3.5,4,T,Per Tax Records,2691,NaN,0,1969,RH2,"RGLR,FNCD","PVDW,PVSW","ATCH,GARG",0,2,4BLK,1BLK,3STR,"PNRM,OCEN,PARK,GRDN"
1,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",San Francisco,CA,1278,La Playa,St,94122,2030,2,SF District 2,2030,2 - Outer Sunset,12/3/15,144,1250000,4/25/16,1075000,9,3.0,4,T,Per Tax Records,2437,0.0689,3000,1947,RM1,RGLR,0,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,0,0
2,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",San Francisco,CA,1278,La Playa,St,94122,2030,2,SF District 2,2030,2 - Outer Sunset,5/18/17,36,1395000,8/17/17,1525000,9,5.0,5,D,Per Architect,2597,0.0689,3000,1947,RM1,RGLR,0,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,0,0
3,-122.50924,37.777330,189.11,"590 48th Ave, San Francisco, CA 94121",San Francisco,CA,590,48th,Ave,94121,1050,1,SF District 1,1050,1 - Outer Richmond,6/27/10,42,725000,8/20/10,715000,5,1.0,2,T,Per Tax Records,1312,NaN,0,1939,RH1,RGLR,"PVDW,PVSW",GARG,0,2,3BLK,1BLK,"ATAC,2STR,FIXR","WATR,OCEN,PARK"
4,-122.50919,37.776695,175.89,"618 48th Ave, San Francisco, CA 94121",San Francisco,CA,618,48th,Ave,94121,1050,1,SF District 1,1050,1 - Outer Richmond,6/7/12,24,1595000,7/13/12,1595000,8,4.0,4,T,Per Tax Records,3307,NaN,0,1951,NaN,RGLR,PVDW,"ATCH,GARG,ATDR,INAC",0,3,4BLK,2BLK,3STR,"PNRM,CTYL,OCEN,PARK"


In [128]:
# Baseline of sale_price
df['sale_price'].mean()

1371454.081312471

In [129]:
# Train, test split on date of 01/01/2018
df['sale_date'] = pd.to_datetime(df['sale_date'], infer_datetime_format=True)
df['year_sold'] = df['sale_date'].dt.year
df['month_sold'] = df['sale_date'].dt.month
df['day_sold'] = df['sale_date'].dt.day

df['on_market_date'] = pd.to_datetime(df['on_market_date'], infer_datetime_format=True)
df['year_on_market'] = df['on_market_date'].dt.year
df['month_on_market'] = df['on_market_date'].dt.month
df['day_on_market'] = df['on_market_date'].dt.day

low_cutoff = 2008
high_cutoff = 2018
train = df[(df['year_sold'] >= low_cutoff) & (df['year_sold'] < high_cutoff)]
test  = df[df['year_sold'] >= high_cutoff]
print(train.shape)
print(test.shape)

(21487, 45)
(2224, 45)


In [130]:
# Wrangle the data for train and test
def engineer_features(X):
  
  # Impute mean for null long/lat/elev based on mean of of neighborhood
  def feature_calc(feature, nid, f_dict):
    if math.isnan(feature):
      if (nid) in f_dict:
        new_feature = f_dict[nid]
        return new_feature
    else:
      return feature
    
    return feature
  
  temp = X[~X['longitude'].isna()].groupby(['nid'])['longitude'].mean()
  long_dict = dict(temp)
  X['longitude'] = X.apply(lambda x: feature_calc(x['longitude'], x['nid'], long_dict), axis=1)
  
  temp = X[~X['latitude'].isna()].groupby(['nid'])['latitude'].mean()
  lat_dict = dict(temp)
  X['latitude'] = X.apply(lambda x: feature_calc(x['latitude'], x['nid'], lat_dict), axis=1)
  
  temp = X[~X['elevation'].isna()].groupby(['nid'])['elevation'].mean()
  elev_dict = dict(temp)
  X['elevation'] = X.apply(lambda x: feature_calc(x['elevation'], x['nid'], elev_dict), axis=1)
  
  # Engineer a cluster feature based on long/lat clusters
  kmeans = KMeans(n_clusters=10, n_jobs=-1)
  X['cluster'] = kmeans.fit_predict(X[['longitude', 'latitude']])
  X['cluster'] = kmeans.predict(X[['longitude', 'latitude']])
  
  X['zip'] = X['zip'].astype(int)
  
  # Fill rooms zero values by adding beds and baths
  def room_calc(rooms_val, beds_val, baths_val):
    if rooms_val == 0:
      total = beds_val + baths_val
    else:
      total = rooms_val
      
    return total
  
  X['rooms'] = X.apply(lambda x: room_calc(x['rooms'], x['beds'], x['baths']), axis=1)
  
  # Fill baths zero values by adding beds and baths
  X.loc[(X['baths'] == 0)] = 1
  
  # Fill house square foot zero values with the average house square footage by bedroom for all single family homes in SF
  averagesf_data = X.groupby('beds').sf.mean()

  # Use average sf by bedroom for each 0 value in each bedroom group up to 9 bedrooms
  for i in range(0, 9): 
    X.loc[(X['sf'] == 0) & (X['beds'] == i), 'sf'] = averagesf_data.loc[i]

  # Use 10,000sf for anything over 9 bedrooms
  X.loc[X['sf'] == 0, 'sf'] = 10000
  X = X.astype({'sf': int})
  
  # Fill lot_sf zero values by using lot_acres to calc
  def lotsf_calc(lotsf_val, lotacres_val):
    if lotsf_val == 0:
      total = lotacres_val * 43560
    else:
      total = lotsf_val
      
    return total
  
  X['lot_sf'] = X.apply(lambda x: lotsf_calc(x['lot_sf'], x['lot_acres']), axis=1)
  
  # Fill lot_acres zero values by using lot_sf to calc
  def lotacres_calc(lotacres_val, lotsf_val):
    if lotacres_val == 0:
      total = lotsf_val / 43560
    else:
      total = lotacres_val
      
    return total
  
  X['lot_sf'] = X.apply(lambda x: lotsf_calc(x['lot_sf'], x['lot_acres']), axis=1)
  X['lot_acres'] = X.apply(lambda x: lotacres_calc(x['lot_acres'], x['lot_sf']), axis=1)
  
  # Features with nan's that should be zero's
  cols_with_nans = ['lot_sf', 'lot_acres', 'year_built']
  for col in cols_with_nans:
    X[col] = X[col].replace(np.nan, 0)
    
  # Features with numeric zero's that should be nan's
  cols_with_czeros = ['lot_desc', 'drive_side', 'parking', 'park_leased',
                      'shopping', 'transportation', 'type', 'views']
  for col in cols_with_czeros:
    X[col] = X[col].replace('0', np.nan)
    
  # Engineer a price_sf column
  X['price_sf'] = X['sale_price'] / X['sf']
  
  # Drop unneeded columns
  unneeded_columns = ['sale_date', 'on_market_date']
  X = X.drop(columns=unneeded_columns)
  
  return X

train = engineer_features(train)
test = engineer_features(test)

train.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view

,longitude,latitude,elevation,full_address,city,state,street_no,street_name,street_suffix,zip,area,district_no,district_desc,nid,neighborhood,cdom,orig_list_price,sale_price,rooms,baths,beds,sf_source,sf_source_decs,sf,lot_acres,lot_sf,year_built,zoning,lot_desc,drive_side,parking,park_leased,num_parking,shopping,transportation,type,views,year_sold,month_sold,day_sold,year_on_market,month_on_market,day_on_market,cluster,price_sf
0,-122.50965,37.780280,200.83,"2645 El Camino Del Mar, San Francisco, CA 94121",San Francisco,CA,2645,El Camino Del Mar,NaN,94121,1050,1,SF District 1,1050,1 - Outer Richmond,4,1095000,1260000,8.0,3.5,4,T,Per Tax Records,2691,0.0000,0.0,1969,RH2,"RGLR,FNCD","PVDW,PVSW","ATCH,GARG",NaN,2,4BLK,1BLK,3STR,"PNRM,OCEN,PARK,GRDN",2013,3,22,2013,3,14,4,468.227425
1,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",San Francisco,CA,1278,La Playa,St,94122,2030,2,SF District 2,2030,2 - Outer Sunset,144,1250000,1075000,9.0,3.0,4,T,Per Tax Records,2437,0.0689,3000.0,1947,RM1,RGLR,NaN,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,NaN,NaN,2016,4,25,2015,12,3,8,441.116126
2,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",San Francisco,CA,1278,La Playa,St,94122,2030,2,SF District 2,2030,2 - Outer Sunset,36,1395000,1525000,9.0,5.0,5,D,Per Architect,2597,0.0689,3000.0,1947,RM1,RGLR,NaN,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,NaN,NaN,2017,8,17,2017,5,18,8,587.216018
3,-122.50924,37.777330,189.11,"590 48th Ave, San Francisco, CA 94121",San Francisco,CA,590,48th,Ave,94121,1050,1,SF District 1,1050,1 - Outer Richmond,42,725000,715000,5.0,1.0,2,T,Per Tax Records,1312,0.0000,0.0,1939,RH1,RGLR,"PVDW,PVSW",GARG,NaN,2,3BLK,1BLK,"ATAC,2STR,FIXR","WATR,OCEN,PARK",2010,8,20,2010,6,27,4,544.969512
4,-122.50919,37.776695,175.89,"618 48th Ave, San Francisco, CA 94121",San Francisco,CA,618,48th,Ave,94121,1050,1,SF District 1,1050,1 - Outer Richmond,24,1595000,1595000,8.0,4.0,4,T,Per Tax Records,3307,0.0000,0.0,1951,NaN,RGLR,PVDW,"ATCH,GARG,ATDR,INAC",NaN,3,4BLK,2BLK,3STR,"PNRM,CTYL,OCEN,PARK",2012,7,13,2012,6,7,4,482.310251


In [131]:
# Train
cutoff = 2017
temp=train.copy()
train = temp[temp['year_sold'] < 2017]
val  = temp[temp['year_sold'] >= 2017]
print(train.shape, val.shape, test.shape)

(19227, 45) (2260, 45) (2224, 45)


In [132]:
train.isna().sum()

longitude              0
latitude               0
elevation              0
full_address           0
city                   0
state                  0
street_no              0
street_name            0
street_suffix       1510
zip                    0
area                   0
district_no            0
district_desc          0
nid                    0
neighborhood           0
cdom                   0
orig_list_price        0
sale_price             0
rooms                  0
baths                  0
beds                   0
sf_source              0
sf_source_decs         0
sf                     0
lot_acres              0
lot_sf                 0
year_built             0
zoning             10309
lot_desc            9022
drive_side         10469
parking             1997
park_leased        12974
num_parking            0
shopping            8466
transportation      6731
type                4480
views               8711
year_sold              0
month_sold             0
day_sold               0


In [133]:
train.dtypes

longitude          float64
latitude           float64
elevation          float64
full_address        object
city                object
state               object
street_no            int64
street_name         object
street_suffix       object
zip                  int64
area                 int64
district_no          int64
district_desc       object
nid                  int64
neighborhood        object
cdom                 int64
orig_list_price      int64
sale_price           int64
rooms              float64
baths              float64
beds                 int64
sf_source           object
sf_source_decs      object
sf                   int64
lot_acres          float64
lot_sf             float64
year_built           int64
zoning              object
lot_desc            object
drive_side          object
parking             object
park_leased         object
num_parking          int64
shopping            object
transportation      object
type                object
views               object
y

In [134]:
# Encode and fit a linear regression model
target = 'sale_price'
features = train.select_dtypes('number').columns.drop(target).tolist()
#features = ['longitude', 'latitude', 'elevation', 'zip', area', 'district_no', 'nid', 'cdom', 'orig_list_price', 
#            'rooms', 'baths', 'beds', 'sf', 'num_parking', 'year_sold', 'month_sold', 'day_sold', 
#            'year_on_market', 'month_on_market', 'day_on_market', 'cluster', 'price_sf']

#features = train.columns.drop(target)
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

pipeline = make_pipeline(
      ce.OrdinalEncoder(), 
      SimpleImputer(strategy='mean'), 
      LinearRegression()
  )

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_val)
  
mae = mean_absolute_error(y_val, y_pred)

X1 = sm.add_constant(X_train)
result = sm.OLS(y_train, X1).fit()

print(f'Test MAE: ${mae:.0f}')
print(result.rsquared, result.rsquared_adj)

Test MAE: $212346
0.8886078670750852 0.8884628327892083


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2389: FutureWarning:

Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.

